# Projeto 2

**1 -** Importando as bibliotecas que serão usadas

In [1]:
import moviepy.editor as mpe
import numpy as np
import matplotlib.pyplot as plt

**2 -** Abrindo o vídeo "video_shrek.mp4" e salvando em uma variável

In [7]:
shrek_inicial = mpe.VideoFileClip("video_shrek.mp4")

**3 -** Invertendo a imagem no eixo a cada 20 segundos

In [ ]:
def inverter_eixo(get_frame, t):
    frame = get_frame(t)
    if (int(t) // 20) % 2 == 1:
        return frame[:, ::-1]
    return frame

shrek_invertido = shrek_inicial.fl(inverter_eixo)

shrek_invertido.ipython_display(width=480, maxduration=162)

**4 -** Diminuindo o som, de forma gradativa, a cada 30 segundos, de forma que os últimos 10 segundos fique em silêncio 

In [ ]:
def reduce_volume(t):
    intervalos = [30, 60, 90, 120, 150]
    volumes = [1.0, 0.8, 0.6, 0.4, 0.2, 0.0]
    
    if np.isscalar(t):
        for i in range(len(intervalos)):
            if t < intervalos[i]:
                return volumes[i]
        return volumes[-1]
    else:
        t = np.array(t)
        out = np.zeros_like(t, dtype=float)
        for i in range(len(intervalos)):
            out = np.where((t >= intervalos[i - 1] if i > 0 else t >= 0) & (t < intervalos[i]), volumes[i], out)
        return np.where(t >= intervalos[-1], volumes[-1], out)


shrek_audio = shrek_invertido.audio
shrek_audio_reduzido = shrek_invertido.audio.fl(lambda gf, t: gf(t) * reduce_volume(t)[:, np.newaxis])
shrek_diminuido = shrek_invertido.set_audio(shrek_audio_reduzido)
shrek_diminuido.ipython_display(width=480, maxduration=162)

**5 -** Após o primeiro minuto, 60 segundos, fazer um corte para o segundo 80

In [ ]:
shrek_cortado_clip1 = shrek_diminuido.subclip(0, 60)
shrek_cortado_clip2 = shrek_diminuido.subclip(80, shrek_diminuido.duration)

shrek_cortado = mpe.concatenate_videoclips([shrek_cortado_clip1, shrek_cortado_clip2])

shrek_cortado.ipython_display(width=480, maxduration=162)

**6 -** Por fim, insira os 20 segundos cortados acima no fim do video

In [ ]:
shrek_cortado_clip = shrek_diminuido.subclip(60, 80)
shrek_final = mpe.concatenate_videoclips([shrek_cortado, shrek_cortado_clip])

shrek_final.write_videofile("shrek_final.mp4")
